## Посимвольная языковая модель.

В первом задании Вам нужно написать и обучить посимвольную нейронную языковую модель для вычисления вероятностей буквенных последовательностей (то есть слов). Такие модели используются в задачах словоизменения и распознавания/порождения звучащей речи. Для обучения модели используйте данные для русского языка из [репозитория](https://github.com/sigmorphon/conll2018/tree/master/task1/surprise).

**В процессе написания Вам нужно решить следующие проблемы:**
    
* как будет выглядеть обучающая выборка; что будет являться признаками, и что - метками классов.
* как сделать так, чтобы модель при предсказании символа учитывала все предыдущие символы слова.
* какие специальные символы нужно использовать.
* как передавать в модель текущее состояние рекуррентной сети

**Результаты:**

* предобработчик данных,
* генератор обучающих данных (батчей),
* обученная модель
* перплексия модели на настроечной выборке
* посимвольные вероятности слов в контрольной выборке

**Дополнительно:**

* дополнительный вход модели (часть речи слова, другие морфологические признаки), влияет ли его добавление на перплексию
* сравнение различных архитектур нейронной сети (FC, RNN, LSTM, QRNN, ...)

Подумайте, какие вспомогательные токены могут быть вам полезны. Выдайте им индексы от `0` до `len(AUXILIARY) - 1`

**План**
- Данные
    - Признаки: набор символов токена, заканчивается токеном END
    - Метки класса: набор символов того же токена, начинается с токена BEGIN
- Для учета всех предыдущих символов, при предсказании следующего символа, дополнительно мы должны передавать на вход предыдущий токен
- Специальные символы
    - BEGIN, END, MASK, UNK
- (???) Как передавать в модель текущее состояние рекуррентной сети

In [27]:
from pathlib import Path

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
# it is better to do all imports at the first cell


In [ ]:
# Uncomment to download data
# !wget https://github.com/sigmorphon/conll2018/blob/master/task1/surprise/russian-train-high
# !wget https://github.com/sigmorphon/conll2018/blob/master/task1/surprise/russian-dev
# !wget https://github.com/sigmorphon/conll2018/blob/master/task1/surprise/russian-covered-test

In [16]:
DATA_PATH = Path('./data')

In [17]:
file_paths = {'train': DATA_PATH/'russian-train-high',
              'dev': DATA_PATH/'russian-dev',
              'test': DATA_PATH/'russian-test'}

In [39]:
df = pd.read_csv(DATA_PATH/'russian-train-high', sep='\t', 
                 header=None, names=['word'], usecols=[0])
df['data_type'] = 'train'
df.head(), df.shape

(            word data_type
 0     валлонский     train
 1  незаконченный     train
 2    истрёпывать     train
 3         личный     train
 4         серьга     train, (10000, 2))

In [13]:
class Vocabulary:
    def __init__(self, token_to_idx=None):
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx
        self._idx_to_token = {idx: token 
                              for token, idx in self._token_to_idx.items()}
    
    def add_token(self, token):
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index
    
    def lookup_token(self, token):
        return self._token_to_idx[token]
    
    def lookup_index(self, index):
        return self._idx_to_token[index]
    
    def __len__(self):
        return len(self._token_to_idx)

In [14]:
class SequenceVocabulary(Vocabulary):
    def __init__(self, token_to_idx=None,
                 unk_token='<UNK>',
                 mask_token='<MASK>',
                 begin_token='<BEGIN>',
                 end_token='<END>'):
        super().__init__(token_to_idx)
        
        self._mask_token = mask_token
        self._unk_token = unk_token
        self._begin_token = begin_token
        self._end_token = end_token
        
        self.mask_index = self.add_token(self._mask_token)
        self.unk_index = self.add_token(self._unk_token)        
        self.begin_index = self.add_token(self._begin_token)        
        self.end_index = self.add_token(self._end_token)
    
    def lookup_token(self, token):
        return self._token_to_idx.get(token, self.unk_index)

In [11]:
class CharLMVectorizer:
    def __init__(self, char_vocab):
        self.char_vocab = char_vocab
        
    def vectorize(self, word):
        indices = [self.char_vocab.begin_index]
        indices.extend(self.char_vocab.lookup_token(token) for token in word)
        indices.append(self.char_vocab.end_index)
        
        vector_length = len(indices) - 1
        
        source_vector = np.empty(vector_length, dtype=np.int64)
        source_indices = indices[:-1]
        source_vector[:len(source_indices)] = source_indices
        source_vector[len(source_indices):] = self.char_vocab.mask_index
        
        target_vector = np.empty(vector_length, dtype=np.int64)
        target_indices = indices[1:]
        target_vector[:len(target_indices)] = source_indices
        target_vector[len(target_indices):] = self.char_vocab.mask_index
        
        return source_vector, target_vector
    
    @classmethod
    def from_dataframe(cls, words_df):
        char_vocab = SequenceVocabulary()
        
        for _, row in words_df.iterrows():
            for char in row['word']:
                char_vocab.add_token(char)
            
        return cls(char_vocab)

In [40]:
class CharLMDataset(Dataset):
    def __init__(self, words_df, vectorizer):
        self.words = words_df
        
        self._vectorizer = vectorizer
        self._max_length = max(map(len, self.words)) + 2 # Why 2?
        
        self.train_df = self.words_df[self.words_df['word'] == 'train']
        self.train_size = len(self.train_df)
        
        self.dev_df = self.words_df[self.words_df['word'] == 'dev']
        self.dev_size = len(self.dev_df)
        
        self.test_df = self.words_df[self.words_df['word'] == 'test']
        self.test_size = len(self.test_df)
        
        self._lookup_dict = {'train': (self.train_df, self.train_size),
                             'dev': (self.dev_df, self.dev_size),
                             'test': (self.test_df, self.test_size)}
        self.set_split('train')
    
    def read_dataset(file_path, data_type):
        df = pd.read_csv(file_path, sep='\t', 
                         header=None, names=['word'], 
                         usecols=[0])
        df['data_type'] = data_type
        return df
    
    @staticmethod
    def load_dataset(files_path):
        dfs_list = []
        
        for data_type, file_path in file_paths.items():
            df = read_dataset(file_path, data_type)
            dfs_list.append(df)

        full_df = pd.concat(dfs_list, axis=0, ignore_index=True)
        
        return full_df
    
    @classmethod
    def make_vectorizer(cls, files_path):
        
        return cls(full_df, CharLMVectorizer.from_dataframe(full_df))
    
    def get_vectorizer(self):
        return self._vectorizer
    
    def set_data_type(self, data_type='train'):
        self._data_type = data_type
        self._data, self._data_size = self._lookup_dict[data_type]
        
    def __len__(self):
        return self._target_size
    
    def __getitem__(self, index):
        row = self._target_df.iloc[index]
        
        source_vector, target_vector = (self._vectorizer.vectorize(row['word'], self._max_seq_length))
        
        return {'source_data': source_vector,
                'target_data': target_vector}
    
    def get_num_batches(self, batch_size):
        return len(self) // batch_size